In [1]:
import mediapipe as mp
import cv2
import numpy as np
import os

# Load the MediaPipe Hands model
mp_hands = mp.solutions.hands.Hands()

def get_landmarks(file_location):
    # Read the image
    image = cv2.imread(file_location)

    # Convert the image to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Process the image and get the landmarks data
    results = mp_hands.process(image_rgb)

    # Extract the landmarks from the results
    landmarks = results.multi_hand_landmarks
    
    # Convert the landmarks to a numpy array
    if landmarks:
        landmarks_np = np.array([[landmark.x, landmark.y, landmark.z] for landmark in landmarks[0].landmark])
    else:
        return None
    
    return landmarks_np

I0000 00:00:1719940545.656459 4463746 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M2 Pro


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1719940545.673585 4464296 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1719940545.680821 4464300 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [2]:
# count the number of subfolders and files in the given 
files = []
folders = []
def count_files(path='.'):
    global file_count, folder_count
    with os.scandir(path) as entries:
        
        for entry in entries:
            # print(entry.path)
            # print(curr_label)
            if entry.is_file():
                files.append(entry.path)
            elif entry.is_dir():
                folders.append(entry.path)
                count_files(entry.path)
        
    return files, folders

count_files("/Users/gvssriram/Desktop/projects-internship/MathNoteX/sign/ASL_Alphabet_Dataset/asl_alphabet_train")

len(set(files)), len(set(folders))

(223075, 29)

In [ ]:
# Initialize the list to store the landmarks
landmarks_list = []
        
# Initialize the list to store the labels
labels = []

global curr_label
curr_label = ""

def getLandmarksAndLabels(path='.'):
    global curr_label
    with os.scandir(path) as entries:
        
        for entry in entries:
            # print(entry.path)
            # print(curr_label)
            if entry.is_file():
                landmarks = get_landmarks(entry.path)
                # print(landmarks)
                if landmarks is not None:
                    landmarks_list.append(landmarks)
                    labels.append(curr_label)
                # else:
                #     print("No landmarks found")
            elif entry.is_dir():
                curr_label = entry.name
                print(curr_label)
                getLandmarksAndLabels(entry.path)
        
    return landmarks_list, labels

x_train, y_train = getLandmarksAndLabels("/Users/gvssriram/Desktop/projects-internship/MathNoteX/sign/ASL_Alphabet_Dataset/asl_alphabet_train")

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


W0000 00:00:1719360603.128705 9646481 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1719360603.134692 9646481 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
/Users/gvssriram/anaconda3/envs/math_note_x/lib/python3.11/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


R
U
I
N
G
Z
T
S
A
F
O
H
del
nothing
space
M
J
C
D
V
Q
X
E
B
K
L
Y
P
W


In [ ]:
type(x_train), type(y_train), type(x_train[0]), type(y_train[0])

(list, list, numpy.ndarray, str)

In [ ]:
# convert the list to numpy array
x_train = np.array(x_train)
y_train = np.array(y_train)

In [ ]:
x_train.shape, y_train.shape

((169090, 21, 3), (169090,))

In [ ]:
np.save('../data/ASL_Alphabet_Dataset_Processed/x_train.npy', x_train)

In [ ]:
np.save('../data/ASL_Alphabet_Dataset_Processed/y_train.npy', y_train)

In [ ]:
# reshape x_train
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1]*x_train.shape[2])
x_train.shape

(169090, 63)

In [ ]:
np.save('../data/ASL_Alphabet_Dataset_Processed/x_train_flat.npy', x_train)

## Load processed numpy image data

In [3]:
# Load numpy arrays
x_train = np.load('../data/ASL_Alphabet_Dataset_Processed/x_train_flat.npy')
y_train = np.load('../data/ASL_Alphabet_Dataset_Processed/y_train.npy')

In [4]:
x_train.shape, y_train.shape

((169090, 63), (169090,))

In [5]:
# one hot encode the labels
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()
y_train = encoder.fit_transform(y_train.reshape(-1,1)).toarray()

In [6]:
y_train.shape

(169090, 29)

In [7]:
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [21]:
# Create a classification model using tensorflow

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from tensorflow.keras.regularizers import l2

model = Sequential()
model.add(Dense(128, input_shape=(x_train.shape[1],), activation='relu', kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(29, activation='softmax'))

model.summary()

opt = tf.keras.optimizers.Adam(learning_rate=0.005)

loss = tf.keras.losses.CategoricalCrossentropy()

metrics = []
metrics.append("categorical_accuracy")

# callbacks

early_stopping = EarlyStopping(monitor='val_loss', patience=10)
model_checkpoint = ModelCheckpoint(filepath='../models/ckpts/asl_alphabet_model.keras', monitor='val_loss', save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=0.0001)
tensorboard = TensorBoard(log_dir='../tflogs')

callbacks = [early_stopping, model_checkpoint, reduce_lr, tensorboard]

model.compile(optimizer=opt, loss=loss, metrics=metrics)

history = model.fit(x_train, y_train, epochs=100, validation_split=0.2, verbose=1, callbacks=callbacks)

/Users/gvssriram/anaconda3/envs/sign_lang_env/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_9 (Dense)                 │ (None, 128)            │         8,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 29)             │           957 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,381 (79.61 KB)

 Trainable params: 19,933 (77.86 KB)

 Non-trainable params: 448 (1.75 KB)

Epoch 1/100
4228/4228 ━━━━━━━━━━━━━━━━━━━━ 5s 845us/step - categorical_accuracy: 0.3986 - loss: 2.4607 - val_categorical_accuracy: 0.0210 - val_loss: 10.5309 - learning_rate: 0.0050
Epoch 2/100
4228/4228 ━━━━━━━━━━━━━━━━━━━━ 4s 845us/step - categorical_accuracy: 0.5074 - loss: 1.8213 - val_categorical_accuracy: 0.0225 - val_loss: 12.6079 - learning_rate: 0.0050
Epoch 3/100
4228/4228 ━━━━━━━━━━━━━━━━━━━━ 3s 801us/step - categorical_accuracy: 0.5270 - loss: 1.7546 - val_categorical_accuracy: 0.0291 - val_loss: 15.1482 - learning_rate: 0.0050
Epoch 4/100
4228/4228 ━━━━━━━━━━━━━━━━━━━━ 3s 804us/step - categorical_accuracy: 0.5271 - loss: 1.7552 - val_categorical_accuracy: 0.0290 - val_loss: 17.3656 - learning_rate: 0.0050
Epoch 5/100
4228/4228 ━━━━━━━━━━━━━━━━━━━━ 3s 797us/step - categorical_accuracy: 0.6046 - loss: 1.4847 - val_categorical_accuracy: 0.0297 - val_loss: 18.2031 - learning_rate: 5.0000e-04
Epoch 6/100
4228/4228 ━━━━━━━━━━━━━━━━━━━━ 3s 801us/step - categorical_accuracy: 0.636

In [22]:
y_pred = model.predict(x_train)

5285/5285 ━━━━━━━━━━━━━━━━━━━━ 1s 272us/step


In [23]:
y_pred[0]

array([7.14140214e-08, 1.12006636e-02, 5.42036898e-04, 5.04569689e-05,
       6.89786684e-05, 9.22070612e-05, 3.77089455e-05, 6.13722345e-03,
       2.93649521e-07, 2.45295246e-06, 5.34611799e-10, 5.64950253e-10,
       3.02420926e-06, 1.39133579e-06, 3.51677636e-06, 5.58958713e-10,
       2.51868827e-04, 5.93809336e-02, 4.73475484e-06, 6.98372742e-06,
       2.97341496e-01, 6.24678195e-01, 5.37512534e-10, 7.41658805e-05,
       5.32230038e-10, 1.15134346e-04, 2.48982087e-06, 1.15135059e-07,
       3.77645642e-06], dtype=float32)

In [24]:
y_pred = np.argmax(y_pred, axis=1)
y_pred

array([21, 17, 20, ..., 21, 21, 21])

In [25]:
y_train_labels = np.argmax(y_train, axis=1)
y_train_labels

array([17, 17, 17, ..., 22, 22, 22])

In [26]:
# Evaluate the model

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

accuracy_score(y_train_labels, y_pred)

0.6728842628186174

In [27]:
print(classification_report(y_train_labels, y_pred))

              precision    recall  f1-score   support

           0       0.79      0.86      0.82      6087
           1       0.80      0.98      0.88      6931
           2       0.84      0.88      0.86      5740
           3       0.60      0.83      0.70      6890
           4       0.83      0.94      0.88      6240
           5       0.95      0.95      0.95      7514
           6       0.67      0.88      0.76      6334
           7       0.68      0.87      0.76      6407
           8       0.68      0.91      0.78      6700
           9       0.65      0.84      0.73      5767
          10       0.00      0.00      0.00      7097
          11       0.00      0.00      0.00      7294
          12       0.68      0.78      0.73      4123
          13       0.86      0.52      0.64      3579
          14       0.84      0.84      0.84      6254
          15       0.00      0.00      0.00      5224
          16       0.78      0.94      0.85      5223
          17       0.61    

/Users/gvssriram/anaconda3/envs/sign_lang_env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/gvssriram/anaconda3/envs/sign_lang_env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/gvssriram/anaconda3/envs/sign_lang_env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

In [ ]:
# Release the MediaPipe Hands model
mp_hands.close()